In [10]:
import cv2
import numpy as np

# Get a VideoCapture object from video and store it in vs
vc = cv2.VideoCapture("April_09_brush_hair_u_nm_np1_ba_goo_0.avi")
# Read first frame
ret, first_frame = vc.read()
# Scale and resize image
resize_dim = 600
max_dim = max(first_frame.shape)
scale = resize_dim/max_dim
first_frame = cv2.resize(first_frame, None, fx=scale, fy=scale)
# Convert to gray scale 
prev_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)


# Create mask
mask = np.zeros_like(first_frame)
# Sets image saturation to maximum
mask[..., 1] = 255


out = cv2.VideoWriter('video.avi',-1,1,(600, 600))

while(vc.isOpened()):
    # Read a frame from video
    ret, frame = vc.read()
    cv2.imshow('porcoddio', frame)
    
    # Convert new frame format`s to gray scale and resize gray frame obtained
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.resize(gray, None, fx=scale, fy=scale)

    # Calculate dense optical flow by Farneback method
    # https://docs.opencv.org/3.0-beta/modules/video/doc/motion_analysis_and_object_tracking.html#calcopticalflowfarneback
    flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, pyr_scale = 0.5, levels = 5, winsize = 11, iterations = 5, poly_n = 5, poly_sigma = 1.1, flags = 0)
    # Compute the magnitude and angle of the 2D vectors
    magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    # Set image hue according to the optical flow direction
    mask[..., 0] = angle * 180 / np.pi / 2
    # Set image value according to the optical flow magnitude (normalized)
    mask[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
    # Convert HSV to RGB (BGR) color representation
    cv2.imshow('porcoddio1', mask)
    rgb = cv2.cvtColor(mask, cv2.COLOR_HSV2BGR)

    # Resize frame size to match dimensions
    frame = cv2.resize(frame, None, fx=scale, fy=scale)
    
    # Open a new window and displays the output frame
    dense_flow = cv2.addWeighted(frame, 1,rgb, 2, 0)
    cv2.imshow("Dense optical flow", dense_flow)
    out.write(dense_flow)
    # Update previous frame
    prev_gray = gray
    # Frame are read by intervals of 1 millisecond. The programs breaks out of the while loop when the user presses the 'q' key
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

KeyboardInterrupt: 

In [1]:
import numpy as np
import cv2 as cv
cap = cv.VideoCapture(cv.samples.findFile("vtest.avi"))
ret, frame1 = cap.read()
prvs = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[..., 1] = 255
while(1):
    ret, frame2 = cap.read()
    if not ret:
        print('No frames grabbed!')
        break
    next = cv.cvtColor(frame2, cv.COLOR_BGR2GRAY)
    flow = cv.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv.cartToPolar(flow[..., 0], flow[..., 1])
    hsv[..., 0] = ang*180/np.pi/2
    hsv[..., 2] = cv.normalize(mag, None, 0, 255, cv.NORM_MINMAX)
    bgr = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)
    cv.imshow('frame2', bgr)
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break
    elif k == ord('s'):
        cv.imwrite('opticalfb.png', frame2)
        cv.imwrite('opticalhsv.png', bgr)
    prvs = next
cv.destroyAllWindows()

# Test 2

In [2]:
import cv2
import numpy as np


def dense_optical_flow(method, video_path, params=[], to_gray=False):
    # read the video
    cap = cv2.VideoCapture(video_path)
    # Read the first frame
    ret, old_frame = cap.read()

    # crate HSV & make Value a constant
    hsv = np.zeros_like(old_frame)
    hsv[..., 1] = 255

    # Preprocessing for exact method
    if to_gray:
        old_frame = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

    while True:
        # Read the next frame
        ret, new_frame = cap.read()
        frame_copy = new_frame
        if not ret:
            break
        # Preprocessing for exact method
        if to_gray:
            new_frame = cv2.cvtColor(new_frame, cv2.COLOR_BGR2GRAY)
        # Calculate Optical Flow
        flow = method(old_frame, new_frame, None, *params)

        # Encoding: convert the algorithm's output into Polar coordinates
        mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        # Use Hue and Saturation to encode the Optical Flow
        hsv[..., 0] = ang * 180 / np.pi / 2
        hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
        # Convert HSV image into BGR for demo
        bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
        cv2.imshow("frame", frame_copy)
        cv2.imshow("optical flow", bgr)
        k = cv2.waitKey(25) & 0xFF
        if k == 27:
            break
        old_frame = new_frame

In [3]:
def lucas_kanade_method(video_path):
    cap = cv2.VideoCapture(video_path)
    # params for ShiTomasi corner detection
    feature_params = dict(maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)
    # Parameters for lucas kanade optical flow
    lk_params = dict(
        winSize=(15, 15),
        maxLevel=2,
        criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03),
    )
    # Create some random colors
    color = np.random.randint(0, 255, (100, 3))
    # Take first frame and find corners in it
    ret, old_frame = cap.read()
    old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
    p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)
    # Create a mask image for drawing purposes
    mask = np.zeros_like(old_frame)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # calculate optical flow
        p1, st, err = cv2.calcOpticalFlowPyrLK(
            old_gray, frame_gray, p0, None, **lk_params
        )
        # Select good points
        good_new = p1[st == 1]
        good_old = p0[st == 1]
        # draw the tracks
        for i, (new, old) in enumerate(zip(good_new, good_old)):
            a, b = new.ravel()
            c, d = old.ravel()
            mask = cv2.line(mask, (a, b), (c, d), color[i].tolist(), 2)
            frame = cv2.circle(frame, (a, b), 5, color[i].tolist(), -1)
        img = cv2.add(frame, mask)
        cv2.imshow("frame", img)
        k = cv2.waitKey(25) & 0xFF
        if k == 27:
            break
        if k == ord("c"):
            mask = np.zeros_like(old_frame)
        # Now update the previous frame and previous points
        old_gray = frame_gray.copy()
        p0 = good_new.reshape(-1, 1, 2)

In [4]:
# "lucaskanade":
video_path = "Increase_stride_length_by_running_stairs__working_step_over_action_speed_development_climb_stairs_f_cm_np1_le_med_1.avi"
lucas_kanade_method(video_path)
cv2.destroyAllWindows()

video_path = "Stabilized/Increase_stride_length_by_running_stairs__working_step_over_action_speed_development_climb_stairs_f_cm_np1_le_med_1.avi"
lucas_kanade_method(video_path)
cv2.destroyAllWindows()

In [11]:
# lucaskanade_dense
method = cv2.optflow.calcOpticalFlowSparseToDense
dense_optical_flow(method, "April_09_brush_hair_u_nm_np1_ba_goo_0.avi", to_gray=True)
cv2.destroyAllWindows()

method = cv2.optflow.calcOpticalFlowSparseToDense
dense_optical_flow(method, "Stabilized/Increase_stride_length_by_running_stairs__working_step_over_action_speed_development_climb_stairs_f_cm_np1_le_med_1.avi", to_gray=True)
cv2.destroyAllWindows()

In [9]:
video_path = "April_09_brush_hair_u_nm_np1_ba_goo_0.avi"
method = cv2.calcOpticalFlowFarneback
params = [0.5, 3, 15, 3, 5, 1.2, 0]  # Farneback's algorithm parameters
dense_optical_flow(method, video_path, params, to_gray=True)
cv2.destroyAllWindows()

video_path = "Stabilized/Increase_stride_length_by_running_stairs__working_step_over_action_speed_development_climb_stairs_f_cm_np1_le_med_1.avi"
method = cv2.calcOpticalFlowFarneback
params = [0.5, 3, 15, 3, 5, 1.2, 0]  # Farneback's algorithm parameters
dense_optical_flow(method, video_path, params, to_gray=True)
cv2.destroyAllWindows()


KeyboardInterrupt: 

In [ ]:
#RLOF
video_path = "Increase_stride_length_by_running_stairs__working_step_over_action_speed_development_climb_stairs_f_cm_np1_le_med_1.avi"
method = cv2.optflow.calcOpticalFlowDenseRLOF
dense_optical_flow(method, video_path)
cv2.destroyAllWindows()

video_path = "Stabilized/Increase_stride_length_by_running_stairs__working_step_over_action_speed_development_climb_stairs_f_cm_np1_le_med_1.avi"
method = cv2.optflow.calcOpticalFlowDenseRLOF
dense_optical_flow(method, video_path)
cv2.destroyAllWindows()



KeyboardInterrupt: 

In [16]:
cv2.destroyAllWindows()


In [5]:
help(cv2.optflow)

Help on module cv2.optflow in cv2:

NAME
    cv2.optflow

FUNCTIONS
    DenseRLOFOpticalFlow_create(...)
        DenseRLOFOpticalFlow_create([, rlofParam[, forwardBackwardThreshold[, gridStep[, interp_type[, epicK[, epicSigma[, epicLambda[, ricSPSize[, ricSLICType[, use_post_proc[, fgsLambda[, fgsSigma[, use_variational_refinement]]]]]]]]]]]]]) -> retval
        .   *    @param rlofParam see optflow::RLOFOpticalFlowParameter
        .        *    @param forwardBackwardThreshold see setForwardBackward
        .        *    @param gridStep see setGridStep
        .        *    @param interp_type see setInterpolation
        .        *    @param epicK see setEPICK
        .        *    @param epicSigma see setEPICSigma
        .        *    @param epicLambda see setEPICLambda
        .        *    @param ricSPSize see setRICSPSize
        .        *    @param ricSLICType see setRICSLICType
        .        *    @param use_post_proc see setUsePostProc
        .        *    @param fgsLambda

In [5]:
import cv2
cv2.__version__
cv2.cuda.getCudaEnabledDeviceCount()

1

In [ ]:
def main():
    #parser = ArgumentParser()
    #parser.add_argument(
    #    "--algorithm",
    #    choices=["farneback", "lucaskanade", "lucaskanade_dense", "rlof"],
    #    required=True,
    #    help="Optical flow algorithm to use",
    #)
    #parser.add_argument(
    #    "--video_path", default="videos/cat.mp4", help="Path to the video",
    #)

    video_path = "videos/people.mp4"
    algorithm = "lucaskanade_dense"
    
    if algorithm == "lucaskanade":
        lucas_kanade_method(video_path)
    elif algorithm == "lucaskanade_dense":
        method = cv2.optflow.calcOpticalFlowSparseToDense
        dense_optical_flow(method, video_path, to_gray=True)
    elif algorithm == "farneback":
        method = cv2.calcOpticalFlowFarneback
        params = [0.5, 3, 15, 3, 5, 1.2, 0]  # Farneback's algorithm parameters
        dense_optical_flow(method, video_path, params, to_gray=True)
    elif algorithm == "rlof":
        method = cv2.optflow.calcOpticalFlowDenseRLOF
        dense_optical_flow(method, video_path)

if __name__ == "__main__":
    main()

In [6]:
import mmcv

: 

In [7]:
if __name__ =='__main__':

    # example: if the data path not setted from args,just manually set them as belows.
    #dataset='ucf101'
    #data_root='/S2/MI/zqj/video_classification/data'
    #data_root=os.path.join(data_root,dataset)

    args=parse_args('')
    # data_root=os.path.join(args.data_root,args.dataset)
    data_root= "C:\\Users\\giorg\\OneDrive - Università degli Studi di Milano-Bicocca\\Laurea Magistrale - Data Science\\directory_progetti\\deep-learning-video-classification\\data\\hmdb51"
    videos_root=os.path.join(data_root,'videos')

    #specify the augments
    
    #num_workers=args.num_workers
    step=args.step
    bound=args.bound
    s_=args.s_
    e_=args.e_
    new_dir=args.new_dir
    mode=args.mode
    #get video list
    video_list,len_videos=get_video_list()
    video_list=video_list[s_:e_]

    len_videos=min(e_-s_,13320-s_) # if we choose the ucf101
    print('find {} videos.').format(len_videos)
    flows_dirs=[video.split('.')[0] for video in video_list]
    print('get videos list done! ')

    pool=Pool(num_workers)
    if mode=='run':
        pool.map(dense_flow,zip(video_list,flows_dirs,[step]*len(video_list),[bound]*len(video_list)))
    else: #mode=='debug
        dense_flow((video_list[0],flows_dirs[0],step,bound))

usage: ipykernel_launcher.py [-h] [--dataset DATASET] [--data_root DATA_ROOT]
                             [--new_dir NEW_DIR] [--num_workers NUM_WORKERS]
                             [--step STEP] [--bound BOUND] [--s_ S_] [--e_ E_]
                             [--mode MODE]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9017 --control=9015 --hb=9014 --Session.signature_scheme="hmac-sha256" --Session.key=b"93d84e49-ec86-42a9-bad2-3832b73dce1d" --shell=9016 --transport="tcp" --iopub=9018 --f=c:\Users\giorg\AppData\Roaming\jupyter\runtime\kernel-v2-37516VFNj5rDf3KIa.json


SystemExit: 2

In [8]:
import mmaction

# 4

In [38]:
def split_to_df(split):
    df = pd.DataFrame(columns=['action', 'title', 'train/test'])
    for action in actions:
        with open(f"./annotations/{action}_test_{split}.txt") as f:
            informations = f.readlines()       
        for info in informations:   #per ogni video della lista dei video di una data azione
            df.loc[len(df)] = action, info.split()[0], int(info.split()[1])
    return df


In [ ]:
split1 = split_to_df("split1")
split2 = split_to_df("split2")
split3 = split_to_df("split3")

In [7]:
videos_root = "C:\\Users\\giorg\\OneDrive - Università degli Studi di Milano-Bicocca\\Laurea Magistrale - Data Science\\directory_progetti\\deep-learning-video-classification\\data\\hmdb51\\videos"
cls_names_list = os.listdir(videos_root)
video_list=[]
video_cls_path_list=[]
for cls_names in os.listdir(videos_root):
    cls_path=os.path.join(videos_root,cls_names)
    for video_ in os.listdir(cls_path):
        video_cls_path = os.path.join(cls_names, video_)
        video_list.append(video_)
        video_cls_path_list.append(video_cls_path)
#video_list.sort()

print(video_cls_path_list[1:5])
print(video_list[1:5])
#print(cls_names_list)
#flows_dirs=[video.split('.')[0] for video in video_list]
#print(flows_dirs)

['brush_hair\\April_09_brush_hair_u_nm_np1_ba_goo_1.avi', 'brush_hair\\April_09_brush_hair_u_nm_np1_ba_goo_2.avi', 'brush_hair\\atempting_to_brush_my_hair_brush_hair_u_nm_np2_le_goo_0.avi', 'brush_hair\\atempting_to_brush_my_hair_brush_hair_u_nm_np2_le_goo_1.avi']
['April_09_brush_hair_u_nm_np1_ba_goo_1.avi', 'April_09_brush_hair_u_nm_np1_ba_goo_2.avi', 'atempting_to_brush_my_hair_brush_hair_u_nm_np2_le_goo_0.avi', 'atempting_to_brush_my_hair_brush_hair_u_nm_np2_le_goo_1.avi']


In [ ]:
import os,sys
import numpy as np
import cv2
from PIL import Image
from multiprocessing import Pool
import argparse
from IPython import embed #to debug
import scipy.misc

from pathlib import Path

import skvideo
skvideo.setFFmpegPath("C:\\ffmpeg\\bin")
import skvideo.io


videos_root = "C:\\Users\\giorg\\OneDrive - Università degli Studi di Milano-Bicocca\\Laurea Magistrale - Data Science\\directory_progetti\\deep-learning-video-classification\\data\\hmdb51\\videos"
skvideo.io.vread(os.path.join(videos_root,'brush_hair\\April_09_brush_hair_u_nm_np1_ba_goo_1' + '.avi'))

videos_root -> dove ci sono le cartelle delle classi (hmdb/videos)
new_dir = cartella generale dell'output (rawframes)
data_root = cartella contenente tutto (hdbmi)
save_dir: save_dir name (always equal to the video id) -> cartella nome del video
param num: the save id, which belongs one of the extracted frames (nome del video)

In [3]:
%%capture --no-stdout

#import warnings
#warnings.filterwarnings('ignore', lineno=38)
#warnings.simplefilter('ignore')

# BACKUP 
import os,sys
import numpy as np
import cv2
from PIL import Image

# sys.modules['__main__'].__file__ = 'ipython'
# import multiprocessing
# from multiprocessing import Pool
from multiprocess import Pool

import argparse
from IPython import embed #to debug
import imageio
from IPython.display import clear_output

import skvideo
skvideo.setFFmpegPath("C:\\ffmpeg\\bin")
import skvideo.io

import scipy.misc

from pathlib import Path

def ToImg(raw_flow,bound):
    '''
    this function scale the input pixels to 0-255 with bi-bound
    :param raw_flow: input raw pixel value (not in 0-255)
    :param bound: upper and lower bound (-bound, bound)
    :return: pixel value scale from 0 to 255
    '''
    flow=raw_flow
    flow[flow>bound]=bound
    flow[flow<-bound]=-bound
    flow-=-bound
    flow*=(255/float(2*bound))
    return flow

def save_flows(flows,image,save_dir,save_dir_cls,num,bound):
    '''
    To save the optical flow images and raw images
    :param flows: contains flow_x and flow_y
    :param image: raw image
    :param save_dir: save_dir name (always equal to the video id)
    :param save_dir_cls: path of save dire name in form of "class//video_name"
    :param num: the save id, which belongs one of the extracted frames
    :param bound: set the bi-bound to flow images
    :return: return 0
    '''
    
    #rescale to 0~255 with the bound setting
    flow_x=ToImg(flows[...,0],bound)
    flow_y=ToImg(flows[...,1],bound)
    if not os.path.exists(os.path.join(data_root,new_dir,save_dir_cls)): #new_dir è il nome della cartella di salvataggio dell'output all'interno di 'data_root', di default è videos
        os.makedirs(os.path.join(data_root,new_dir,save_dir_cls))

    #save the image
    save_img=os.path.join(data_root,new_dir,save_dir_cls,'img_{:05d}.jpg'.format(num)) # prefisso immagini: img_
    # scipy.misc.imsave(save_img,image)
    # cv2.imwrite(save_img, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))  
    imageio.imwrite(save_img,image)  

    #save the flows
    save_x=os.path.join(data_root,new_dir,save_dir_cls,'flow_x_{:05d}.jpg'.format(num)) # prefisso optical flow x: flow_x_
    save_y=os.path.join(data_root,new_dir,save_dir_cls,'flow_y_{:05d}.jpg'.format(num)) # prefisso optical flow y: flow_y_
    flow_x_img=Image.fromarray(flow_x)
    flow_y_img=Image.fromarray(flow_y)
    #scipy.misc.imsave(save_x,flow_x_img)
    imageio.imwrite(save_x,flow_x_img)
    #scipy.misc.imsave(save_y,flow_y_img)
    imageio.imwrite(save_y,flow_y_img)
    return 0

def dense_flow(augs):
    '''
    To extract dense_flow images
    :param augs:the detailed augments:
        video_name: the video name which is like: 'v_xxxxxxx',if different ,please have a modify.
        save_dir: the destination path's final direction name. video_name (without .ext)
        save_dir_cls: added by me. class_name//video_name (without .ext)
        step: num of frames between each two extracted frames
        bound: bi-bound parameter
    :return: no returns
    '''
    print('Dense Flow function activated')
    video_name,save_dir,save_dir_cls,step,bound=augs #flow_dir viene passato come save_dir (essenzialmente è il nome del video senza avi)
    # video_path=os.path.join(videos_root,video_name.split('_')[1],video_name) # CAMBIATO TOLTO LO SPLIT
    video_path=os.path.join(videos_root,save_dir_cls + '.avi') 
    # provide two video-read methods: cv2.VideoCapture() and skvideo.io.vread(), both of which need ffmpeg support

    # videocapture=cv2.VideoCapture(video_path)
    # if not videocapture.isOpened():
    #     print 'Could not initialize capturing! ', video_name
    #     exit()
    
    print("Parsed argouments:")
    print(args)
    print(f'Videos root: {videos_root}')
    
    try:
        videocapture=skvideo.io.vread(video_path)
    except:
        print('{} read error! ').format(video_name)
        return 0
    print(video_name)
    # if extract nothing, exit!
    if videocapture.sum()==0:
        print('Could not initialize capturing'),video_name
        exit()
    else: print(f'Inizialize capturing of {video_path}')
    len_frame=len(videocapture)
    frame_num=0
    image,prev_image,gray,prev_gray=None,None,None,None
    num0=0
    while True:
        #frame=videocapture.read()
        if num0>=len_frame:
            break
        frame=videocapture[num0]
        num0+=1
        if frame_num==0:
            image=np.zeros_like(frame)
            gray=np.zeros_like(frame)
            prev_gray=np.zeros_like(frame)
            prev_image=frame
            prev_gray=cv2.cvtColor(prev_image,cv2.COLOR_RGB2GRAY)
            frame_num+=1
            # to pass the out of stepped frames
            step_t=step
            while step_t>1:
                #frame=videocapture.read()
                num0+=1
                step_t-=1
            continue

        image=frame
        gray=cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
        frame_0=prev_gray
        frame_1=gray
        ##default choose the tvl1 algorithm
        dtvl1=cv2.optflow.createOptFlow_DualTVL1()
        flowDTVL1=dtvl1.calc(frame_0,frame_1,None)
        print(f'Salvataggio frame {frame_num}')
        save_flows(flowDTVL1,image,save_dir,save_dir_cls,frame_num,bound) #this is to save flows and img.
        prev_gray=gray
        prev_image=image
        frame_num+=1
        # to pass the out of stepped frames
        step_t=step
        while step_t>1:
            #frame=videocapture.read()
            num0+=1
            step_t-=1
    clear_output(wait=True)


'''
Funzione che trova 1) La lista di tutti i nomi dei video 2) La lunghezza di tale lista (quanti video ci sono)
'''
def get_video_list():
    cls_names_list = os.listdir(videos_root)
    video_list=[]
    video_cls_path_list=[]
    for cls_names in os.listdir(videos_root):
        cls_path=os.path.join(videos_root,cls_names)
        for video_ in os.listdir(cls_path):
            video_cls_path = os.path.join(cls_names, video_)
            video_list.append(video_)
            video_cls_path_list.append(video_cls_path)
    #video_list.sort()
    return video_list,len(video_list),video_cls_path_list


#  step         | 0    | right - left (0 for img, non-0 for flow) / step is 1, ie flow of adjacent frames
#  bound        | 32   | maximum of optical flow
# START_IDX (s_): The start index of extracted frames.

def parse_args():
    parser = argparse.ArgumentParser(description="densely extract the video frames and optical flows")
    parser.add_argument('--dataset',default='hmdb51',type=str,help='set the dataset name, to find the data path')
    parser.add_argument('--data_root',default='/n/zqj/video_classification/data',type=str)
    parser.add_argument('--new_dir',default='rawframes',type=str) # cambiato il default prima era 'flows'
    parser.add_argument('--num_workers',default=4,type=int,help='num of workers to act multi-process')
    parser.add_argument('--step',default=1,type=int,help='gap frames')
    parser.add_argument('--bound',default=15,type=int,help='set the maximum of optical flow') # cambiato il default prima era '15'
    # nel codice otterremo la lista di tutti i video e poi una sottolista da essi
    parser.add_argument('--s_',default=0,type=int,help='start id') # indica l'indice nella lista da cui parte la sottolista
    parser.add_argument('--e_',default=6766,type=int,help='end id') # indica l'indice in cui finisce la sottolista (default = 6766 ovvero il numero di video)
    parser.add_argument('--mode',default='run',type=str,help='set \'run\' if debug done, otherwise, set debug')
    args = parser.parse_args('') # mettendo come argomento le '' vengono passati gli argomenti base
    return args

if __name__ == '__main__':

    # example: if the data path not setted from args,just manually set them as belows.
    #dataset='ucf101'
    #data_root='/S2/MI/zqj/video_classification/data'
    #data_root=os.path.join(data_root,dataset)

    args=parse_args()
    
    cwd = Path.cwd()
    dataset= "hmdb51"
    data_root = os.path.join(Path.cwd(),dataset)
    videos_root= os.path.join(data_root,'videos')

    #specify the augments
    num_workers=args.num_workers
    step=args.step
    bound=args.bound
    s_=args.s_
    e_=args.e_
    #new_dir=args.new_dir
    new_dir = "rawframes"
    #mode= args.mode
    mode = "run"
    
    print("Parsed argouments:")
    print(args)
    print(f'Videos root: {videos_root}')
    
    #get video list
    video_list,len_videos,video_cls_path_list=get_video_list()
    video_list=video_list[s_:e_] #seleziona la sottolista
    video_cls_path_list=video_cls_path_list[s_:e_] #seleziona sottolista di save/flow_dir

    len_videos=min(e_-s_,6766-s_) # numero totale di video selezionati
    print(f'find {len_videos} videos.')
    #print('find {} videos.').format(len_videos)
    flows_dirs=[video.split('.')[0] for video in video_list] # genera una lista dei video senza il '.avi' (nome della cartella)
    flows_dirs_cls =[video.split('.')[0] for video in video_cls_path_list] # genera una lista dei video senza il '.avi' per i path
    
    print('get videos list done! ')

    # Parallelizzazione
    pool = Pool(num_workers) # setta il numero di processi paralleli
    #print([len(video_list),len(flows_dirs),len(flows_dirs_cls),len([step]*len(video_list)),len([bound]*len(video_list))])
    
    if mode=='run': #modalità run
        print('Run mode activated')
        pool.map(dense_flow,zip(video_list,flows_dirs,flows_dirs_cls,[step]*len(video_list),[bound]*len(video_list)))
    else: #mode=='debug
        print('Debug mode activated')
        dense_flow((video_list[0],flows_dirs[0],flows_dirs_cls[0],step,bound))
    pool.close()
    pool.join()
    clear_output(wait=True)
    print('FINISHED')

Parsed argouments:
Namespace(bound=20, data_root='/n/zqj/video_classification/data', dataset='hmdb51', e_=6766, mode='run', new_dir='rawframes', num_workers=4, s_=0, step=1)
Videos root: c:\Users\giorg\OneDrive - Università degli Studi di Milano-Bicocca\Laurea Magistrale - Data Science\directory_progetti\deep-learning-video-classification\data\hmdb51\videos
find 6766 videos.
get videos list done! 
Run mode activated


NameError: name 'os' is not defined

In [9]:
from pathlib import Path
cwd = Path.cwd()
print(cwd)
print(os.listdir(os.path.join(cwd,"hmdb51"+'.avi')))

c:\Users\giorg\OneDrive - Università degli Studi di Milano-Bicocca\Laurea Magistrale - Data Science\directory_progetti\deep-learning-video-classification\data


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'c:\\Users\\giorg\\OneDrive - Università degli Studi di Milano-Bicocca\\Laurea Magistrale - Data Science\\directory_progetti\\deep-learning-video-classification\\data\\hmdb51.avi'

In [1]:
%%capture --no-stdout

#import warnings
#warnings.filterwarnings('ignore', lineno=38)
#warnings.simplefilter('ignore')

# BACKUP 
import os,sys
import numpy as np
import cv2
from PIL import Image

sys.modules['__main__'].__file__ = 'ipython'
from multiprocessing import Pool

import argparse
from IPython import embed #to debug
import imageio
from IPython.display import clear_output

import skvideo
skvideo.setFFmpegPath("C:\\ffmpeg\\bin")
import skvideo.io

import scipy.misc

from pathlib import Path

def ToImg(raw_flow,bound):
    '''
    this function scale the input pixels to 0-255 with bi-bound
    :param raw_flow: input raw pixel value (not in 0-255)
    :param bound: upper and lower bound (-bound, bound)
    :return: pixel value scale from 0 to 255
    '''
    flow=raw_flow
    flow[flow>bound]=bound
    flow[flow<-bound]=-bound
    flow-=-bound
    flow*=(255/float(2*bound))
    return flow

def save_flows(flows,image,save_dir,save_dir_cls,num,bound):
    '''
    To save the optical flow images and raw images
    :param flows: contains flow_x and flow_y
    :param image: raw image
    :param save_dir: save_dir name (always equal to the video id)
    :param save_dir_cls: path of save dire name in form of "class//video_name"
    :param num: the save id, which belongs one of the extracted frames
    :param bound: set the bi-bound to flow images
    :return: return 0
    '''
    
    #rescale to 0~255 with the bound setting
    flow_x=ToImg(flows[...,0],bound)
    flow_y=ToImg(flows[...,1],bound)
    if not os.path.exists(os.path.join(data_root,new_dir,save_dir_cls)): #new_dir è il nome della cartella di salvataggio dell'output all'interno di 'data_root', di default è videos
        os.makedirs(os.path.join(data_root,new_dir,save_dir_cls))

    #save the image
    save_img=os.path.join(data_root,new_dir,save_dir_cls,'img_{:05d}.jpg'.format(num)) # prefisso immagini: img_
    # scipy.misc.imsave(save_img,image)
    # cv2.imwrite(save_img, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))  
    imageio.imwrite(save_img,image)  

    #save the flows
    save_x=os.path.join(data_root,new_dir,save_dir_cls,'flow_x_{:05d}.jpg'.format(num)) # prefisso optical flow x: flow_x_
    save_y=os.path.join(data_root,new_dir,save_dir_cls,'flow_y_{:05d}.jpg'.format(num)) # prefisso optical flow y: flow_y_
    flow_x_img=Image.fromarray(flow_x)
    flow_y_img=Image.fromarray(flow_y)
    #scipy.misc.imsave(save_x,flow_x_img)
    imageio.imwrite(save_x,flow_x_img)
    #scipy.misc.imsave(save_y,flow_y_img)
    imageio.imwrite(save_y,flow_y_img)
    return 0

def dense_flow(augs):
    '''
    To extract dense_flow images
    :param augs:the detailed augments:
        video_name: the video name which is like: 'v_xxxxxxx',if different ,please have a modify.
        save_dir: the destination path's final direction name. video_name (without .ext)
        save_dir_cls: added by me. class_name//video_name (without .ext)
        step: num of frames between each two extracted frames
        bound: bi-bound parameter
    :return: no returns
    '''
    print('Dense Flow function activated')
    video_name,save_dir,save_dir_cls,step,bound=augs #flow_dir viene passato come save_dir (essenzialmente è il nome del video senza avi)
    # video_path=os.path.join(videos_root,video_name.split('_')[1],video_name) # CAMBIATO TOLTO LO SPLIT
    video_path=os.path.join(videos_root,save_dir_cls + '.avi') 
    # provide two video-read methods: cv2.VideoCapture() and skvideo.io.vread(), both of which need ffmpeg support

    # videocapture=cv2.VideoCapture(video_path)
    # if not videocapture.isOpened():
    #     print 'Could not initialize capturing! ', video_name
    #     exit()
    
    print("Parsed argouments:")
    print(args)
    print(f'Videos root: {videos_root}')
    
    try:
        videocapture=skvideo.io.vread(video_path)
    except:
        print('{} read error! ').format(video_name)
        return 0
    print(video_name)
    # if extract nothing, exit!
    if videocapture.sum()==0:
        print('Could not initialize capturing'),video_name
        exit()
    else: print(f'Inizialize capturing of {video_path}')
    len_frame=len(videocapture)
    frame_num=0
    image,prev_image,gray,prev_gray=None,None,None,None
    num0=0
    while True:
        #frame=videocapture.read()
        if num0>=len_frame:
            break
        frame=videocapture[num0]
        num0+=1
        if frame_num==0:
            image=np.zeros_like(frame)
            gray=np.zeros_like(frame)
            prev_gray=np.zeros_like(frame)
            prev_image=frame
            prev_gray=cv2.cvtColor(prev_image,cv2.COLOR_RGB2GRAY)
            frame_num+=1
            # to pass the out of stepped frames
            step_t=step
            while step_t>1:
                #frame=videocapture.read()
                num0+=1
                step_t-=1
            continue

        image=frame
        gray=cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
        frame_0=prev_gray
        frame_1=gray
        ##default choose the tvl1 algorithm
        dtvl1=cv2.optflow.createOptFlow_DualTVL1()
        flowDTVL1=dtvl1.calc(frame_0,frame_1,None)
        print(f'Salvataggio frame {frame_num}')
        save_flows(flowDTVL1,image,save_dir,save_dir_cls,frame_num,bound) #this is to save flows and img.
        prev_gray=gray
        prev_image=image
        frame_num+=1
        # to pass the out of stepped frames
        step_t=step
        while step_t>1:
            #frame=videocapture.read()
            num0+=1
            step_t-=1
    clear_output(wait=True)


'''
Funzione che trova 1) La lista di tutti i nomi dei video 2) La lunghezza di tale lista (quanti video ci sono)
'''
def get_video_list():
    cls_names_list = os.listdir(videos_root)
    video_list=[]
    video_cls_path_list=[]
    for cls_names in os.listdir(videos_root):
        cls_path=os.path.join(videos_root,cls_names)
        for video_ in os.listdir(cls_path):
            video_cls_path = os.path.join(cls_names, video_)
            video_list.append(video_)
            video_cls_path_list.append(video_cls_path)
    #video_list.sort()
    return video_list,len(video_list),video_cls_path_list


#  step         | 0    | right - left (0 for img, non-0 for flow) / step is 1, ie flow of adjacent frames
#  bound        | 32   | maximum of optical flow
# START_IDX (s_): The start index of extracted frames.

def parse_args():
    parser = argparse.ArgumentParser(description="densely extract the video frames and optical flows")
    parser.add_argument('--dataset',default='hmdb51',type=str,help='set the dataset name, to find the data path')
    parser.add_argument('--data_root',default='/n/zqj/video_classification/data',type=str)
    parser.add_argument('--new_dir',default='rawframes',type=str) # cambiato il default prima era 'flows'
    parser.add_argument('--num_workers',default=4,type=int,help='num of workers to act multi-process')
    parser.add_argument('--step',default=1,type=int,help='gap frames')
    parser.add_argument('--bound',default=20,type=int,help='set the maximum of optical flow') # cambiato il default prima era '15'
    # nel codice otterremo la lista di tutti i video e poi una sottolista da essi
    parser.add_argument('--s_',default=0,type=int,help='start id') # indica l'indice nella lista da cui parte la sottolista
    parser.add_argument('--e_',default=6766,type=int,help='end id') # indica l'indice in cui finisce la sottolista (default = 6766 ovvero il numero di video)
    parser.add_argument('--mode',default='run',type=str,help='set \'run\' if debug done, otherwise, set debug')
    args = parser.parse_args('') # mettendo come argomento le '' vengono passati gli argomenti base
    return args

# example: if the data path not setted from args,just manually set them as belows.
#dataset='ucf101'
#data_root='/S2/MI/zqj/video_classification/data'
#data_root=os.path.join(data_root,dataset)

args=parse_args()

cwd = Path.cwd()
dataset= "hmdb51"
data_root = os.path.join(Path.cwd(),dataset)
videos_root= os.path.join(data_root,'videos')

#specify the augments
num_workers=args.num_workers
step=args.step
bound=args.bound
s_=args.s_
e_=args.e_
#new_dir=args.new_dir
new_dir = "rawframes"
#mode= args.mode
mode = "run"

print("Parsed argouments:")
print(args)
print(f'Videos root: {videos_root}')

#get video list
video_list,len_videos,video_cls_path_list=get_video_list()
video_list=video_list[s_:e_] #seleziona la sottolista
video_cls_path_list=video_cls_path_list[s_:e_] #seleziona sottolista di save/flow_dir

len_videos=min(e_-s_,6766-s_) # numero totale di video selezionati
print(f'find {len_videos} videos.')
#print('find {} videos.').format(len_videos)
flows_dirs=[video.split('.')[0] for video in video_list] # genera una lista dei video senza il '.avi' (nome della cartella)
flows_dirs_cls =[video.split('.')[0] for video in video_cls_path_list] # genera una lista dei video senza il '.avi' per i path

print('get videos list done! ')

# Parallelizzazione
pool = Pool(num_workers) # setta il numero di processi paralleli
#print([len(video_list),len(flows_dirs),len(flows_dirs_cls),len([step]*len(video_list)),len([bound]*len(video_list))])

if mode=='run': #modalità run
    print('Run mode activated')
    pool.map(dense_flow,zip(video_list,flows_dirs,flows_dirs_cls,[step]*len(video_list),[bound]*len(video_list)))
else: #mode=='debug
    print('Debug mode activated')
    dense_flow((video_list[0],flows_dirs[0],flows_dirs_cls[0],step,bound))
pool.close()
pool.join()
clear_output(wait=True)
print('FINISHED')

Parsed argouments:
Namespace(bound=20, data_root='/n/zqj/video_classification/data', dataset='hmdb51', e_=6766, mode='run', new_dir='rawframes', num_workers=4, s_=0, step=1)
Videos root: c:\Users\giorg\OneDrive - Università degli Studi di Milano-Bicocca\Laurea Magistrale - Data Science\directory_progetti\deep-learning-video-classification\data\hmdb51\videos
find 6766 videos.
get videos list done! 
Run mode activated
